In [1]:
import os
#os.chdir('/Users/rupesh.karn/Desktop/WorkPart-1/UNSW-NB15 Dataset')
import pandas as pd
import numpy as np
import csv
import os
import time
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import pylab
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
pylab.rcParams['figure.figsize'] = (16.0, 5.0)
import sklearn

# Read in the training CSV file
print "Reading Training csv file."
df1 = pd.read_csv("UNSW_NB15_training-set.csv")
df1.drop('label', axis=1, inplace=True)

#One hot encoding the string variables
obj_df1=df1
obj_df1["proto"] = obj_df1["proto"].astype('category')
obj_df1["service"] = obj_df1["service"].astype('category')
obj_df1["state"] = obj_df1["state"].astype('category')
obj_df1["proto_cat"] = obj_df1["proto"].cat.codes
obj_df1["service_cat"] = obj_df1["service"].cat.codes
obj_df1["state_cat"] = obj_df1["state"].cat.codes

obj_df1["proto"] = obj_df1["proto_cat"]
obj_df1["service"] = obj_df1["service_cat"]
obj_df1["state"] = obj_df1["state_cat"]

obj_df1.drop('proto_cat', axis=1, inplace=True)
obj_df1.drop('service_cat', axis=1, inplace=True)
obj_df1.drop('state_cat', axis=1, inplace=True)

Y_train_all_attacks = obj_df1["attack_cat"]
X_train = obj_df1.values[:,:-1]

#Normalizing train set
for j in range(0,43):
    maximum = max(X_train[:,j])
    for i in range(0,len(X_train)):
        X_train[i,j] = round(X_train[i,j]/maximum,3)

#Make a catagorical cloumn for each type of label in trainset
obj_df1=pd.get_dummies(obj_df1, columns=["attack_cat"])
Y_train_each_attach = obj_df1.values[:,-10:]
        
# Read in the testing CSV file 
print "Reading Testing csv file."
df2 = pd.read_csv("UNSW_NB15_testing-set.csv")
df2.drop('label', axis=1, inplace=True)

#One hot encoding the string variables
obj_df2=df2
obj_df2["proto"] = obj_df2["proto"].astype('category')
obj_df2["service"] = obj_df2["service"].astype('category')
obj_df2["state"] = obj_df2["state"].astype('category')
obj_df2["proto_cat"] = obj_df2["proto"].cat.codes
obj_df2["service_cat"] = obj_df2["service"].cat.codes
obj_df2["state_cat"] = obj_df2["state"].cat.codes

obj_df2["proto"] = obj_df2["proto_cat"]
obj_df2["service"] = obj_df2["service_cat"]
obj_df2["state"] = obj_df2["state_cat"]

obj_df2.drop('proto_cat', axis=1, inplace=True)
obj_df2.drop('service_cat', axis=1, inplace=True)
obj_df2.drop('state_cat', axis=1, inplace=True)

Y_test_all_attacks = obj_df2["attack_cat"]
X_test = obj_df2.values[:,:-1]

#Normalizing test set
for j in range(0,43):
    maximum = max(X_train[:,j])
    for i in range(0,len(X_test)):
        X_test[i,j] = round(X_test[i,j]/maximum,3)
        
#Make a catagorical cloumn for each type of label in testset
obj_df2=pd.get_dummies(obj_df2, columns=["attack_cat"])
Y_test_each_attach = obj_df2.values[:,-10:]

cleanup_nums = {"Worms":0, "Shellcode":1, "Reconnaissance":2, "Normal":3, "Generic":4, "Fuzzers":5, "Exploits":6, "DoS":7, "Backdoor":8, "Analysis":9}
Y_train_all_attacks.replace(cleanup_nums,inplace=True)
Y_test_all_attacks.replace(cleanup_nums,inplace=True)
print('Trainset size: ',X_train.shape,Y_train_all_attacks.shape,'Testset size: ',X_test.shape,Y_test_all_attacks.shape)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Reading Training csv file.
Reading Testing csv file.
('Trainset size: ', (175341, 43), (175341,), 'Testset size: ', (82332, 43), (82332,))


In [2]:
#Binary classification for each attack-type
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Model for binary classification
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=43, kernel_initializer='random_uniform', activation='relu'))
    model.add(Dense(30, kernel_initializer='random_uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='random_uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

attack_name = obj_df1.columns[-10:]
model_weights_save = []
# evaluate model with standardized dataset
for attack_type in range(len(attack_name)):
    Y_train = Y_train_each_attach[:,-attack_type]
    Y_test = Y_test_each_attach[:,-attack_type]
    estimator = KerasClassifier(build_fn=create_baseline, batch_size=100, verbose=0)
    kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print('Attack:',attack_name[attack_type].split('_')[-1])
    print("Train Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

    results = cross_val_score(estimator, X_test, Y_test, cv=kfold)
    print("Test Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    print('-----------------')

Using TensorFlow backend.


('Attack:', 'Analysis')
Train Results: 98.88% (0.00%)
Test Results: 99.18% (0.00%)
-----------------
('Attack:', 'Backdoor')
Train Results: 99.93% (0.00%)
Test Results: 99.95% (0.00%)
-----------------
('Attack:', 'DoS')
Train Results: 99.35% (0.00%)
Test Results: 99.54% (0.00%)
-----------------
('Attack:', 'Exploits')
Train Results: 94.00% (0.02%)
Test Results: 95.75% (0.00%)
-----------------
('Attack:', 'Fuzzers')
Train Results: 92.31% (0.02%)
Test Results: 55.85% (9.81%)
-----------------
('Attack:', 'Generic')
Train Results: 97.30% (0.04%)
Test Results: 77.08% (0.00%)
-----------------
('Attack:', 'Normal')
Train Results: 89.99% (0.03%)
Test Results: 92.64% (0.00%)
-----------------
('Attack:', 'Reconnaissance')
Train Results: 85.11% (0.35%)
Test Results: 86.48% (0.00%)
-----------------
('Attack:', 'Shellcode')
Train Results: 93.00% (0.00%)
Test Results: 95.03% (0.00%)
-----------------
('Attack:', 'Worms')
Train Results: 99.00% (0.00%)
Test Results: 99.29% (0.00%)
-------------

In [8]:
#Multilabel classification
from keras.layers import Dense, Dropout, Activation
import keras.initializers

hidden_units = 100

# Model for multiclass classification
def create_baseline():
    model = Sequential()
    keras.initializers.he_normal(seed=None)
    model.add(Dense(60, input_dim=43,  activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(60,  activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(10,  activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
print(create_baseline().summary())
estimator = KerasClassifier(build_fn=create_baseline, batch_size=100, verbose=0)
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_train, Y_train_all_attacks, cv=kfold)
print("Train Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

results = cross_val_score(estimator, X_test, Y_test_all_attacks, cv=kfold)
print("Test Results: %.2f%% (%.2f%%) \n" % (results.mean()*100, results.std()*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_178 (Dense)            (None, 60)                2640      
_________________________________________________________________
dropout_39 (Dropout)         (None, 60)                0         
_________________________________________________________________
dense_179 (Dense)            (None, 60)                3660      
_________________________________________________________________
dropout_40 (Dropout)         (None, 60)                0         
_________________________________________________________________
dense_180 (Dense)            (None, 10)                610       
Total params: 6,910
Trainable params: 6,910
Non-trainable params: 0
_________________________________________________________________
None
Train Results: 70.83% (0.04%)
Test Results: 44.94% (0.00%) 

